<div align="center"><img src="encabezado1.png" width=1000, height=50></div>


# Proyecto Final

## Presentado por: Jaribeth Bernal

<br>

### Problema:
El cliente  esta tiendo pérdidas por producto vencidos en diferentes tiendas, sus ventas a penas mantienen el punto de equilibrio, requieren enfocar sus recursos en los puntos de venta, horarios y de target correcto.

In [2]:
import pandas as pd
import datetime
import sqlite3

La data trabajada consta de 3 data frame:
- RawData Ventas: información principal de ventas por cliente, fecha , hora y tienda, de los últimos 2 meses del 2021
- catalogo_clientes: información de variables referentes al cliente
- catalogo_tienda: información de variables referentes a la tienda

## 1. catalogo_clientes

Incluimos este catálogo desde un archivo csv a un df, con la función *read_csv*, luego damos un primer vistazo al df con *head()*


In [3]:
df_1 = pd.read_csv('catalogo_clientes.csv', index_col=0)
df_1.head()

,Cliente,Genero,F_Nac_00000000000
1,1472560,Femenino,10/21/1983
20,1182892,Masculino,02/18/1988
27,976052,Femenino,03/09/1980
34,993468,Femenino,08/23/1979
73,637674,Femenino,06/21/1964


Revisamos el tamaño de la matriz con el método *shape*, que nos muestra la cantidad de filas y columnas en nuestro df

In [49]:
df_1.shape

(197936, 3)

Podemos observar, que el índice no esta en un orden secuencial y la llave para este df sería la columna de **Cliente**, se procede a crear un índice secuencial, que nos permita reflejar la cantidad de linea del df, esto utilizan el método *reset_index()*

In [5]:
df_1.reset_index()

,index,Cliente,Genero,F_Nac_00000000000
0,1,1472560,Femenino,10/21/1983
1,20,1182892,Masculino,02/18/1988
2,27,976052,Femenino,03/09/1980
3,34,993468,Femenino,08/23/1979
4,73,637674,Femenino,06/21/1964
...,...,...,...,...
197931,46233,2635964,Femenino,02/29/1984
197932,46234,2738582,Masculino,09/11/1940
197933,46235,2735476,Femenino,07/29/1947
197934,46236,208066,Masculino,01/09/1975


Procedemos a eliminar la columna con el indice anterior con *drop=True*

In [6]:
df_1.reset_index(drop=True)

,Cliente,Genero,F_Nac_00000000000
0,1472560,Femenino,10/21/1983
1,1182892,Masculino,02/18/1988
2,976052,Femenino,03/09/1980
3,993468,Femenino,08/23/1979
4,637674,Femenino,06/21/1964
...,...,...,...
197931,2635964,Femenino,02/29/1984
197932,2738582,Masculino,09/11/1940
197933,2735476,Femenino,07/29/1947
197934,208066,Masculino,01/09/1975


Procedemos a guardar los ajustes en un nuevo df

In [7]:
df_users= df_1.reset_index(drop=True)
df_users

,Cliente,Genero,F_Nac_00000000000
0,1472560,Femenino,10/21/1983
1,1182892,Masculino,02/18/1988
2,976052,Femenino,03/09/1980
3,993468,Femenino,08/23/1979
4,637674,Femenino,06/21/1964
...,...,...,...
197931,2635964,Femenino,02/29/1984
197932,2738582,Masculino,09/11/1940
197933,2735476,Femenino,07/29/1947
197934,208066,Masculino,01/09/1975


Ahora revisamos si existen nulos, con la función vectorizada *isna()*, a esta le adicionamos la agregación *sum()*, y de esta manera nos va a sumarizar los nulos encontrados para cada columna

In [8]:
df_users.isna().sum()

Cliente              0
Genero               0
F_Nac_00000000000    0
dtype: int64

Validamos que los formatos de las columnas tengan coherencia y nos dejen manipular los datos más adelante, utilizando *types*

In [9]:
df_users.dtypes

Cliente               int64
Genero               object
F_Nac_00000000000    object
dtype: object

Se observa que la columna **F_Nac_00000000000**, debería ser un campo de tipo date, por lo que se requiere hace la modifacion del tipo para esta columna a tipo *datetime*, de esta manera mas adelante podemos generar datos adicionales con esta columas.

Utilizamos un diccionario, para almacenar el nuevo tipo de dato y luego de hacer el ajuste, revisamos que el cambio sea el correcto, por lo que utilizamos nuevamente *dtypes*

In [10]:
diccionario_de_conversion = {
    "F_Nac_00000000000":"datetime64[ns]",
}

In [11]:
df_users = df_users.astype(diccionario_de_conversion)
df_users.dtypes

Cliente                       int64
Genero                       object
F_Nac_00000000000    datetime64[ns]
dtype: object

Adicional al cambio de tipo de datos, vamos a colocar un nombre que sea visualmente mas apropiado, esto con la funcion *rename* y con *head* validamos que se haya aplicado correctamente

In [12]:
df_users = df_users.rename(columns={'F_Nac_00000000000': 'F_Nac',})
df_users.head()

,Cliente,Genero,F_Nac
0,1472560,Femenino,1983-10-21
1,1182892,Masculino,1988-02-18
2,976052,Femenino,1980-03-09
3,993468,Femenino,1979-08-23
4,637674,Femenino,1964-06-21


Con la columna **F_Nac**, podemos obtener mas datos para nuestro df, por lo que creamos 2 funciones que nos permitan obtener la edad y hacer una clasificación por rango en base a esta.

<br>
Creamos la funcion age, para obtener la edad, tomando en cuenta el día de hoy como referencia para el calculo y luego la funcion rango_edad para hacer la clasificación, tomando en cuenta la función anterior

In [13]:
def age(value):
     # Calcula la edad tomando en cuenta el día actual 
    today = datetime.date.today()
    age = today.year - value.year - ((today.month, today.day) < (value.month, value.day))
    return age

def rango_edad(value):
    #Segmenta la edad en rangos
    if age(value) <= 24:
        return '18 a 24'
    elif age(value) <= 31:
        return '25 a 31'
    elif age(value) <= 38:
        return '32 a 38'   
    elif age(value) <= 45:
        return '39 a 45'
    elif age(value) <= 52:
        return '46 a 52'
    elif age(value) <= 59:
        return '53 a 59'     
    else:
        return '60 +'        


Validamos en efecto se esten haciendo los calculos de forma correcta para ambas funciones

In [14]:
df_users.head()

,Cliente,Genero,F_Nac
0,1472560,Femenino,1983-10-21
1,1182892,Masculino,1988-02-18
2,976052,Femenino,1980-03-09
3,993468,Femenino,1979-08-23
4,637674,Femenino,1964-06-21


In [15]:
df_users['F_Nac'].transform(age)

0         38
1         33
2         41
3         42
4         57
          ..
197931    37
197932    81
197933    74
197934    47
197935    37
Name: F_Nac, Length: 197936, dtype: int64

In [16]:
df_users['F_Nac'].transform(rango_edad)

0         32 a 38
1         32 a 38
2         39 a 45
3         39 a 45
4         53 a 59
           ...   
197931    32 a 38
197932       60 +
197933       60 +
197934    46 a 52
197935    32 a 38
Name: F_Nac, Length: 197936, dtype: object

Incluimos las nuevas columnas al df **Edad** y **Rango_Edades**, utilizando la función *lambda*

In [17]:
df_users['Edad'] = df_users.apply(lambda x: age(x['F_Nac']), axis=1)

df_users['Rango_Edades'] = df_users.apply(lambda x: rango_edad(x['F_Nac']), axis=1) 


In [18]:
df_users.head()

,Cliente,Genero,F_Nac,Edad,Rango_Edades
0,1472560,Femenino,1983-10-21,38,32 a 38
1,1182892,Masculino,1988-02-18,33,32 a 38
2,976052,Femenino,1980-03-09,41,39 a 45
3,993468,Femenino,1979-08-23,42,39 a 45
4,637674,Femenino,1964-06-21,57,53 a 59


## 2. catalogo_tiendas

Incluimos este conjunto de datos, utilizando *sqlite*, cramos la conexión y un cursor

In [19]:
bd = "proyecto.sqlite3"
conn = sqlite3.connect(bd) # Crea una conexión a la base de datos SQLite
cur = conn.cursor() # Con la conexión, crea un objeto cursor
cur

Realizamos una primera consulta sobre la tabla a cargar con *execute* y luego extramos la informacion con *fetchall* que nos genera la lista de tuplas

In [20]:
cur.execute("SELECT * FROM catalogo_tienda")

In [21]:
resultados1 = cur.fetchall() #Si recuerdas el último ejemplo, 
#fetchall nos regresa una lista de tuplas. Cada tupla representa una fila
resultados1

[(1,
  'Tienda 1',
  'Panamá Metro',
  'Supermercado',
  'Plaza Concordia, Vía España, Panama City',
  4907),
 (4,
  'Tienda 10',
  'Panamá Metro',
  'Conveniencia',
  'Centro Comercial La Mañana, Panamá',
  4932),
 (14, 'Tienda 11', 'Colón', 'Supermercado', 'Calle 13, Panamá', 4906),
 (3,
  'Tienda 12',
  'Panamá Metro',
  'Conveniencia',
  'Centro Comercial Villa Lucre',
  4976),
 (28,
  'Tienda 13',
  'Panamá Oeste',
  'Supermercado',
  'Plaza Hotel Las Lajas, Panamá',
  4943),
 (18, 'Tienda 14', 'Panamá Metro', 'Supermercado', 'Calle 50, Panamá', 4918),
 (8,
  'Tienda 15',
  'Panamá Oeste',
  'Supermercado',
  'Blvd. Costa Verde, La Chorrera, Panamá',
  4929),
 (11,
  'Tienda 16',
  'Colón',
  'Conveniencia',
  'Calle 7ma 8, Colón, Panamá',
  4927),
 (10,
  'Tienda 17',
  'Panamá Metro',
  'Supermercado',
  'Plaza Las Americas, Panamá',
  4964),
 (24,
  'Tienda 18',
  'Panamá Metro',
  'Supermercado',
  'Vía Centenario, Panamá',
  4973),
 (27,
  'Tienda 19',
  'Panamá Metro',
  'Su

Para transformar esta lista de tuplas, utilizamos una listas de comprensión de manera que nos permita hacer la carga al df

In [22]:
columnas = [ x [0] for x in cur.description] 
columnas

['field', 'Tienda', 'Zona', 'Formato', 'Dirección', 'IDTienda']

In [23]:
df_3=pd.DataFrame(resultados1, columns=columnas)

df_3.head()

,field,Tienda,Zona,Formato,Dirección,IDTienda
0,1,Tienda 1,Panamá Metro,Supermercado,"Plaza Concordia, Vía España, Panama City",4907
1,4,Tienda 10,Panamá Metro,Conveniencia,"Centro Comercial La Mañana, Panamá",4932
2,14,Tienda 11,Colón,Supermercado,"Calle 13, Panamá",4906
3,3,Tienda 12,Panamá Metro,Conveniencia,Centro Comercial Villa Lucre,4976
4,28,Tienda 13,Panamá Oeste,Supermercado,"Plaza Hotel Las Lajas, Panamá",4943


Borramos el indíce original y guardamos en un nuevo df.

In [24]:
df_tiendas = df_3.drop(columns=['field'])
df_tiendas.head()

,Tienda,Zona,Formato,Dirección,IDTienda
0,Tienda 1,Panamá Metro,Supermercado,"Plaza Concordia, Vía España, Panama City",4907
1,Tienda 10,Panamá Metro,Conveniencia,"Centro Comercial La Mañana, Panamá",4932
2,Tienda 11,Colón,Supermercado,"Calle 13, Panamá",4906
3,Tienda 12,Panamá Metro,Conveniencia,Centro Comercial Villa Lucre,4976
4,Tienda 13,Panamá Oeste,Supermercado,"Plaza Hotel Las Lajas, Panamá",4943


## 3. ventas_fecha_hora

Genenramos la nueva consulta sobre la tabla de ventas que es nuestra tabla principal y generamos la lista de tuplas

In [25]:
cur.execute("SELECT * FROM ventas_fecha_hora")

In [26]:
resultados2 = cur.fetchall() 
resultados2

[(4913, 'Productos de pasteleria fresca', 2839860, 'Friday', 20, 0.438),
 (4973, 'Productos de pasteleria fresca', 2278694, 'Friday', 19, 0.276),
 (4930, 'Productos de pasteleria fresca', 666906, 'Tuesday', 9, 0.69),
 (4938, 'Productos de pasteleria fresca', 2282738, 'Monday', 14, 0.456),
 (4979, 'Productos de pasteleria fresca', 2441840, 'Sunday', 10, 0.676),
 (4951, 'Productos de pasteleria fresca', 2829068, 'Saturday', 1, 0.158),
 (4917, 'Productos de pasteleria fresca', 74394, 'Sunday', 19, 0.458),
 (4905, 'Productos de pasteleria fresca', 2573298, 'Saturday', 18, 0.276),
 (4963, 'Productos de pasteleria fresca', 92892, 'Thursday', 20, 0.43),
 (4966, 'Productos de pasteleria fresca', 358222, 'Friday', 16, 0.704),
 (4968, 'Productos de pasteleria fresca', 2651924, 'Thursday', 22, 0.662),
 (4938, 'Productos de pasteleria fresca', 2257148, 'Tuesday', 9, 0.38),
 (4964, 'Productos de pasteleria fresca', 1912990, 'Friday', 21, 0.438),
 (4913, 'Productos de pasteleria fresca', 2886150, 'T

Utilizamos la lista de comprensión para hacer la carga y luego guardamos el resultado en el nuevo df

In [27]:
columnas = [ y [0] for y in cur.description] 
columnas

['IDTienda', 'Categoria', 'Cliente', 'DiaSemana', 'HORA', 'SALES']

In [28]:
df_ventas=pd.DataFrame(resultados2, columns=columnas)

df_ventas.head()

,IDTienda,Categoria,Cliente,DiaSemana,HORA,SALES
0,4913,Productos de pasteleria fresca,2839860,Friday,20,0.438
1,4973,Productos de pasteleria fresca,2278694,Friday,19,0.276
2,4930,Productos de pasteleria fresca,666906,Tuesday,9,0.690
3,4938,Productos de pasteleria fresca,2282738,Monday,14,0.456
4,4979,Productos de pasteleria fresca,2441840,Sunday,10,0.676


Ya con los 3 dataframe limpios, procederemos a unirlos con el método *merge*

Primero vamos a unir los dataframe de ventas con el catalogo de clientes, mediante la columna de **IDTienda**

In [29]:
pd.merge(left=df_ventas, 
         right=df_tiendas, 
         how='left', 
         left_on='IDTienda', 
         right_on='IDTienda')

,IDTienda,Categoria,Cliente,DiaSemana,HORA,SALES,Tienda,Zona,Formato,Dirección
0,4913,Productos de pasteleria fresca,2839860,Friday,20,0.438,Tienda 8,Colón,Hipermercado,"Sabanitas, Panamá"
1,4973,Productos de pasteleria fresca,2278694,Friday,19,0.276,Tienda 18,Panamá Metro,Supermercado,"Vía Centenario, Panamá"
2,4930,Productos de pasteleria fresca,666906,Tuesday,9,0.690,Tienda 4,Panamá Oeste,Hipermercado,"La Chorrera, Calle del Puerto, La Chorrera, Pa..."
3,4938,Productos de pasteleria fresca,2282738,Monday,14,0.456,Tienda 29,Chiriquí / Bocas,Supermercado,"Avenida Francisco Clark, David, Chiriquí, Panamá"
4,4979,Productos de pasteleria fresca,2441840,Sunday,10,0.676,Tienda 25,Panamá Metro,Supermercado,"Brisas del Golf, frente a Centro Comercial Las..."
...,...,...,...,...,...,...,...,...,...,...
543251,4966,Productos de pasteleria fresca,146872,Saturday,16,0.458,Tienda 27,Panamá Metro,Supermercado,"Av 6 Sur, Panamá"
543252,4963,Productos de pasteleria fresca,1768060,Sunday,15,3.178,Tienda 2,Panamá Oeste,Supermercado,"Carr.Panamericana, Villas del Carrizal, Arraiján"
543253,4918,Productos de pasteleria fresca,2294032,Saturday,13,0.234,Tienda 14,Panamá Metro,Supermercado,"Calle 50, Panamá"
543254,4952,Productos de pasteleria fresca,2028344,Sunday,20,0.468,Tienda 5,Panamá Metro,Hipermercado,"Av. 12 de Octubre, Panamá"


Luego de visualizar que la información esta correctamente asignada, guardamos el resultado

In [30]:
df_5 = pd.merge(left=df_ventas, 
         right=df_tiendas, 
         how='left', 
         left_on='IDTienda', 
         right_on='IDTienda')
df_5

,IDTienda,Categoria,Cliente,DiaSemana,HORA,SALES,Tienda,Zona,Formato,Dirección
0,4913,Productos de pasteleria fresca,2839860,Friday,20,0.438,Tienda 8,Colón,Hipermercado,"Sabanitas, Panamá"
1,4973,Productos de pasteleria fresca,2278694,Friday,19,0.276,Tienda 18,Panamá Metro,Supermercado,"Vía Centenario, Panamá"
2,4930,Productos de pasteleria fresca,666906,Tuesday,9,0.690,Tienda 4,Panamá Oeste,Hipermercado,"La Chorrera, Calle del Puerto, La Chorrera, Pa..."
3,4938,Productos de pasteleria fresca,2282738,Monday,14,0.456,Tienda 29,Chiriquí / Bocas,Supermercado,"Avenida Francisco Clark, David, Chiriquí, Panamá"
4,4979,Productos de pasteleria fresca,2441840,Sunday,10,0.676,Tienda 25,Panamá Metro,Supermercado,"Brisas del Golf, frente a Centro Comercial Las..."
...,...,...,...,...,...,...,...,...,...,...
543251,4966,Productos de pasteleria fresca,146872,Saturday,16,0.458,Tienda 27,Panamá Metro,Supermercado,"Av 6 Sur, Panamá"
543252,4963,Productos de pasteleria fresca,1768060,Sunday,15,3.178,Tienda 2,Panamá Oeste,Supermercado,"Carr.Panamericana, Villas del Carrizal, Arraiján"
543253,4918,Productos de pasteleria fresca,2294032,Saturday,13,0.234,Tienda 14,Panamá Metro,Supermercado,"Calle 50, Panamá"
543254,4952,Productos de pasteleria fresca,2028344,Sunday,20,0.468,Tienda 5,Panamá Metro,Hipermercado,"Av. 12 de Octubre, Panamá"


Ahora hacemos merge para terminar de completar la informacion con la tabla de clientes y luego guardamos resultados

In [31]:
pd.merge(left=df_5, 
         right=df_users, 
         how='left', 
         left_on='Cliente', 
         right_on='Cliente')

,IDTienda,Categoria,Cliente,DiaSemana,HORA,SALES,Tienda,Zona,Formato,Dirección,Genero,F_Nac,Edad,Rango_Edades
0,4913,Productos de pasteleria fresca,2839860,Friday,20,0.438,Tienda 8,Colón,Hipermercado,"Sabanitas, Panamá",Masculino,1982-06-08,39,39 a 45
1,4973,Productos de pasteleria fresca,2278694,Friday,19,0.276,Tienda 18,Panamá Metro,Supermercado,"Vía Centenario, Panamá",Femenino,1991-09-26,30,25 a 31
2,4930,Productos de pasteleria fresca,666906,Tuesday,9,0.690,Tienda 4,Panamá Oeste,Hipermercado,"La Chorrera, Calle del Puerto, La Chorrera, Pa...",Masculino,1978-12-10,43,39 a 45
3,4938,Productos de pasteleria fresca,2282738,Monday,14,0.456,Tienda 29,Chiriquí / Bocas,Supermercado,"Avenida Francisco Clark, David, Chiriquí, Panamá",Femenino,1988-12-28,33,32 a 38
4,4979,Productos de pasteleria fresca,2441840,Sunday,10,0.676,Tienda 25,Panamá Metro,Supermercado,"Brisas del Golf, frente a Centro Comercial Las...",Femenino,1973-09-17,48,46 a 52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543251,4966,Productos de pasteleria fresca,146872,Saturday,16,0.458,Tienda 27,Panamá Metro,Supermercado,"Av 6 Sur, Panamá",Masculino,1958-06-13,63,60 +
543252,4963,Productos de pasteleria fresca,1768060,Sunday,15,3.178,Tienda 2,Panamá Oeste,Supermercado,"Carr.Panamericana, Villas del Carrizal, Arraiján",Masculino,1981-05-03,40,39 a 45
543253,4918,Productos de pasteleria fresca,2294032,Saturday,13,0.234,Tienda 14,Panamá Metro,Supermercado,"Calle 50, Panamá",Masculino,1988-11-24,33,32 a 38
543254,4952,Productos de pasteleria fresca,2028344,Sunday,20,0.468,Tienda 5,Panamá Metro,Hipermercado,"Av. 12 de Octubre, Panamá",Masculino,1972-05-15,49,46 a 52


In [108]:
df_analizar = pd.merge(left=df_5, 
         right=df_users, 
         how='left', 
         left_on='Cliente', 
         right_on='Cliente')
df_analizar

,IDTienda,Categoria,Cliente,DiaSemana,HORA,SALES,Tienda,Zona,Formato,Dirección,Genero,F_Nac,Edad,Rango_Edades
0,4913,Productos de pasteleria fresca,2839860,Friday,20,0.438,Tienda 8,Colón,Hipermercado,"Sabanitas, Panamá",Masculino,1982-06-08,39,39 a 45
1,4973,Productos de pasteleria fresca,2278694,Friday,19,0.276,Tienda 18,Panamá Metro,Supermercado,"Vía Centenario, Panamá",Femenino,1991-09-26,30,25 a 31
2,4930,Productos de pasteleria fresca,666906,Tuesday,9,0.690,Tienda 4,Panamá Oeste,Hipermercado,"La Chorrera, Calle del Puerto, La Chorrera, Pa...",Masculino,1978-12-10,43,39 a 45
3,4938,Productos de pasteleria fresca,2282738,Monday,14,0.456,Tienda 29,Chiriquí / Bocas,Supermercado,"Avenida Francisco Clark, David, Chiriquí, Panamá",Femenino,1988-12-28,33,32 a 38
4,4979,Productos de pasteleria fresca,2441840,Sunday,10,0.676,Tienda 25,Panamá Metro,Supermercado,"Brisas del Golf, frente a Centro Comercial Las...",Femenino,1973-09-17,48,46 a 52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543251,4966,Productos de pasteleria fresca,146872,Saturday,16,0.458,Tienda 27,Panamá Metro,Supermercado,"Av 6 Sur, Panamá",Masculino,1958-06-13,63,60 +
543252,4963,Productos de pasteleria fresca,1768060,Sunday,15,3.178,Tienda 2,Panamá Oeste,Supermercado,"Carr.Panamericana, Villas del Carrizal, Arraiján",Masculino,1981-05-03,40,39 a 45
543253,4918,Productos de pasteleria fresca,2294032,Saturday,13,0.234,Tienda 14,Panamá Metro,Supermercado,"Calle 50, Panamá",Masculino,1988-11-24,33,32 a 38
543254,4952,Productos de pasteleria fresca,2028344,Sunday,20,0.468,Tienda 5,Panamá Metro,Hipermercado,"Av. 12 de Octubre, Panamá",Masculino,1972-05-15,49,46 a 52


Validamos nuesto dataframe final

In [109]:
df_analizar.isna().sum()

IDTienda        0
Categoria       0
Cliente         0
DiaSemana       0
HORA            0
SALES           0
Tienda          0
Zona            0
Formato         0
Dirección       0
Genero          0
F_Nac           0
Edad            0
Rango_Edades    0
dtype: int64

### Ahora con nuestros datos listos para ser analizados, que sigue?

Procedamos a buscar dónde se encuentran concentradas la mayor parte de las ventas, esto se conoce como *Estrategia de segmento de Mercado*

Esto para dirigir todos sus esfuerzos de marketing y estrategias para especializarse en atender a ese grupo de mercado.

<div align="center"><img src="Estrategia-de-segmento-de-mercado.png" width=400, height=100></div>

Inicialmente analizamos, cuales son las zonas en las que se concentra la mayor parte de la venta, esto lo podemos hacer agrupando el campo de **Zona** y sumando el valor de **Sales** para cada una, con *group by* y *sum*

In [110]:
df_analizar.groupby('Zona')['SALES'].sum()

Zona
Chiriquí / Bocas     11718.212
Colón                31545.576
Panamá Metro        177255.740
Panamá Oeste         58917.588
Prov. Centrales      18073.398
Name: SALES, dtype: float64

Podemos observar que donde mayorente se concentra la venta, es la Zona de *Panamá Metro*, procedamos entonces a tener en cuenta como se dividen a su vez las ventas por hora dentro de la misma.

Para esto procedemos inicialmente a filtrar el dataframe principal solo a los registros que pertenecen a *Panamá Metro*

In [117]:
in_PmaMetro = df_analizar['Zona'] == 'Panamá Metro'
df_PmaMetro = df_analizar[in_PmaMetro]
df_PmaMetro

,IDTienda,Categoria,Cliente,DiaSemana,HORA,SALES,Tienda,Zona,Formato,Dirección,Genero,F_Nac,Edad,Rango_Edades
1,4973,Productos de pasteleria fresca,2278694,Friday,19,0.276,Tienda 18,Panamá Metro,Supermercado,"Vía Centenario, Panamá",Femenino,1991-09-26,30,25 a 31
4,4979,Productos de pasteleria fresca,2441840,Sunday,10,0.676,Tienda 25,Panamá Metro,Supermercado,"Brisas del Golf, frente a Centro Comercial Las...",Femenino,1973-09-17,48,46 a 52
5,4951,Productos de pasteleria fresca,2829068,Saturday,1,0.158,Tienda 22,Panamá Metro,Supermercado,"Sta. María Business District, frente al Hombre...",Femenino,1970-01-25,52,46 a 52
9,4966,Productos de pasteleria fresca,358222,Friday,16,0.704,Tienda 27,Panamá Metro,Supermercado,"Av 6 Sur, Panamá",Masculino,1950-05-28,71,60 +
10,4968,Productos de pasteleria fresca,2651924,Thursday,22,0.662,Tienda 7,Panamá Metro,Supermercado,"Las Cumbres, Milla 8",Femenino,1981-11-17,40,39 a 45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543249,4918,Productos de pasteleria fresca,1326694,Friday,13,0.598,Tienda 14,Panamá Metro,Supermercado,"Calle 50, Panamá",Masculino,1981-05-22,40,39 a 45
543250,11196,Productos de pasteleria fresca,3075180,Friday,9,0.150,Tienda 21,Panamá Metro,Supermercado,"Brisas Norte, Las Cumbres",Femenino,1983-04-26,38,32 a 38
543251,4966,Productos de pasteleria fresca,146872,Saturday,16,0.458,Tienda 27,Panamá Metro,Supermercado,"Av 6 Sur, Panamá",Masculino,1958-06-13,63,60 +
543253,4918,Productos de pasteleria fresca,2294032,Saturday,13,0.234,Tienda 14,Panamá Metro,Supermercado,"Calle 50, Panamá",Masculino,1988-11-24,33,32 a 38


In [118]:
df_PmaMetro.groupby('DiaSemana')['SALES'].sum()

DiaSemana
Friday       28388.620
Monday       25627.512
Saturday     24441.028
Sunday       23075.262
Thursday     25274.402
Tuesday      25243.674
Wednesday    25205.242
Name: SALES, dtype: float64

In [36]:
#cur.close()

In [37]:
#conn.close()